# Testing

In [1]:
import jax.numpy as jnp
from jax import jacfwd

from geometry.manifolds import LorentzFinslerManifold, nEllipsoid, nSphere, RiemannianNavigation

from geometry.geodesic import GEORCE
from geometry.tacking import SequentialOptimizationBFGS, SequentialOptimizationADAM

In [2]:
def Falpha(t,x1,v):
    
    x,y = v[0], v[1]
    
    return (1.0*(2+1/2*jnp.sin(t))**2*jnp.sin(1/4*t*jnp.pi)*y \
            +1.0*(3/2+3/8*jnp.sin(t))**2*jnp.cos(1/4*t*jnp.pi)*x \
                +((2+1/2*jnp.sin(t))**4*(3/2+3/8*jnp.sin(t))**2*y**2 \
                  +(2+1/2*jnp.sin(t))**2*(3/2+3/8*jnp.sin(t))**4*x**2-\
                      1.00*(2+1/2*jnp.sin(t))**2*(3/2+3/8*jnp.sin(t))**2*jnp.cos(1/4*t*jnp.pi)**2*y**2 \
                          +2.00*(2+1/2*jnp.sin(t))**2*(3/2+3/8*jnp.sin(t))**2*jnp.cos(1/4*t*jnp.pi) \
                              *jnp.sin(1/4*t*jnp.pi)*x*y-1.00*(2+1/2*jnp.sin(t))**2*\
                                  (3/2+3/8*jnp.sin(t))**2*jnp.sin(1/4*t*jnp.pi)**2*x**2)**(1/2)) \
        /((2+1/2*jnp.sin(t))**2*(3/2+3/8*jnp.sin(t))**2-1.00*(2+1/2*jnp.sin(t))**2\
          *jnp.sin(1/4*t*jnp.pi)**2-1.00*(3/2+3/8*jnp.sin(t))**2*jnp.cos(1/4*t*jnp.pi)**2)

In [3]:
Malpha = LorentzFinslerManifold(F=Falpha)

In [4]:
z0 = jnp.zeros(2)
zT = jnp.array([5*jnp.pi, 0.0])

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [5]:
N = 200
zt = jnp.linspace(0,1,N).reshape(-1,1)*(zT-z0)+z0

In [6]:
def z_cont(s):
    
    return (zT-z0)*s+z0

def dz_cont(s):
    
    return jacfwd(z_cont)(s)

In [7]:
def disc_integral(F, zt):
    
    t = 0.0
    for i in range(len(zt)-1):
        t += F(t,zt[i], zt[i+1]-zt[i]) 
    
    return t

print(disc_integral(Falpha, zt))

8.730320325424202


In [8]:
def cont_integral(F, z_fun, dz_fun, T=1.0, N=100):
    
    h = T/N
    t = 0.0
    for i in range(N):
        t += F(t,z_fun(t), dz_fun(t))*h
    
    return t

print(cont_integral(Falpha, z_cont, dz_cont, T=1.0, N=N))

8.730271630571146


In [9]:
tack_metrics = [Falpha]

In [10]:
Geodesic = GEORCE(Malpha, init_fun=None, T=100, tol=1e-4, max_iter=1000, line_search_params={'rho': 0.5})

In [11]:
Tacking = SequentialOptimizationBFGS(tack_metrics, init_fun=None, max_iter=1000, 
                                     tol=1e-4, T=100, sub_iter=1, line_search_params={'rho': 0.5})

In [12]:
Geodesic.dim = 2
Geodesic.t0 = 0.0
Geodesic.z0 = z0
Geodesic.zT = zT

In [13]:
Geodesic.time_fun(zt)

Array([0.07893449, 0.15485686, 0.22806652, 0.29881847, 0.36733173,
       0.43379591, 0.4983763 , 0.56121795, 0.62244888, 0.68218272,
       0.74052078, 0.79755384, 0.85336357, 0.90802371, 0.96160111,
       1.01415657, 1.06574551, 1.11641868, 1.16622259, 1.21520004,
       1.26339048, 1.31083036, 1.35755345, 1.40359107, 1.44897236,
       1.49372446, 1.53787272, 1.58144086, 1.62445108, 1.66692426,
       1.70888   , 1.7503368 , 1.79131211, 1.83182244, 1.87188342,
       1.9115099 , 1.950716  , 1.98951517, 2.02792024, 2.06594348,
       2.10359666, 2.14089105, 2.17783753, 2.21444653, 2.25072814,
       2.28669213, 2.32234793, 2.35770472, 2.3927714 , 2.42755664,
       2.46206889, 2.49631641, 2.53030727, 2.56404938, 2.59755049,
       2.63081822, 2.66386006, 2.69668338, 2.72929544, 2.76170342,
       2.79391441, 2.8259354 , 2.85777334, 2.8894351 , 2.92092749,
       2.95225729, 2.98343121, 3.01445594, 3.04533815, 3.07608445,
       3.10670147, 3.1371958 , 3.16757403, 3.19784276, 3.22800